<a href="https://colab.research.google.com/github/CaioHenri99/Sistema_de_Recomendacao/blob/main/RecomendacaoDeFilmes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recomendação de Filmes

In [60]:
#@title Importação de Bibliotecas
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [61]:
#@title Carregando Dataset
arquivos = [
    'Action.csv', 'Adventure.csv', 'Animation.csv', 'Biography.csv',
    'Comedy.csv', 'Crime.csv', 'Drama.csv', 'Fantasy.csv',
    'History.csv', 'Horror.csv', 'Sci-Fi.csv'
]

dataframes = []
for arquivo in arquivos:
    if os.path.exists(arquivo):
        try:
            df_temp = pd.read_csv(arquivo)
            dataframes.append(df_temp)
        except:
            print(f"Erro ao ler {arquivo}")

if not dataframes:
    raise ValueError("Nenhum arquivo foi carregado!")

In [62]:
#@title  Junta tudo em um único DataFrame
filmes = pd.concat(dataframes, ignore_index=True)

In [63]:
#@title Removendo Duplicatas
filmes.drop_duplicates(subset=['name', 'year'], keep='first', inplace=True)
print(f"Base de dados criada com {len(filmes)} filmes únicos!")

Base de dados criada com 753 filmes únicos!


In [64]:
#@title Analisando as Informações de cada Coluna
filmes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 753 entries, 0 to 1099
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          753 non-null    object 
 1   year          753 non-null    int64  
 2   movie_rated   753 non-null    object 
 3   run_length    753 non-null    object 
 4   genres        753 non-null    object 
 5   release_date  753 non-null    object 
 6   rating        753 non-null    float64
 7   num_raters    753 non-null    int64  
 8   num_reviews   753 non-null    int64  
 9   review_url    753 non-null    object 
dtypes: float64(1), int64(3), object(6)
memory usage: 64.7+ KB


In [65]:
#@title Visualizando o Dataset
filmes.head(20)

,name,year,movie_rated,run_length,genres,release_date,rating,num_raters,num_reviews,review_url
0,The Dark Knight,2008,PG-13,2h 32min,Action; Crime; Drama;,18 July 2008 (USA),9.0,2224522,6836,https://www.imdb.com/title/tt0468569/reviews/_...
1,Inception,2010,PG-13,2h 28min,Action; Adventure; Sci-Fi;,16 July 2010 (USA),8.8,1981675,3820,https://www.imdb.com/title/tt1375666/reviews/_...
2,The Matrix,1999,R,2h 16min,Action; Sci-Fi;,31 March 1999 (USA),8.7,1619761,4281,https://www.imdb.com/title/tt0133093/reviews/_...
3,The Lord of the Rings: The Fellowship of the Ring,2001,PG-13,2h 58min,Action; Adventure; Drama;,19 December 2001 (USA),8.8,1609165,5365,https://www.imdb.com/title/tt0120737/reviews/_...
4,The Dark Knight Rises,2012,PG-13,2h 44min,Action; Adventure;,20 July 2012 (USA),8.4,1470329,2979,https://www.imdb.com/title/tt1345836/reviews/_...
5,Gladiator,2000,R,2h 35min,Action; Adventure; Drama;,5 May 2000 (USA),8.5,1297718,2636,https://www.imdb.com/title/tt0172495/reviews/_...
6,Batman Begins,2005,PG-13,2h 20min,Action; Adventure;,15 June 2005 (USA),8.2,1269027,2885,https://www.imdb.com/title/tt0372784/reviews/_...
7,The Avengers,2012,PG-13,2h 23min,Action; Adventure; Sci-Fi;,4 May 2012 (USA),8.0,1236048,1929,https://www.imdb.com/title/tt0848228/reviews/_...
8,Star Wars: Episode IV - A New Hope,1977,PG,2h 1min,Action; Adventure; Fantasy;,25 May 1977 (USA),8.6,1197132,1834,https://www.imdb.com/title/tt0076759/reviews/_...
9,Star Wars: Episode V - The Empire Strikes Back,1980,PG,2h 4min,Action; Adventure; Fantasy;,20 June 1980 (USA),8.7,1124834,1214,https://www.imdb.com/title/tt0080684/reviews/_...


In [66]:
#@title Verificando o Tamanho do Dataset
filmes.shape

(753, 10)

In [67]:
#@title Limpando a coluna de gêneros para o algoritmo entender

# O CountVectorizer vai transformar "Action; Crime; Drama" em vetores matemáticos
filmes['genres_clean'] = filmes['genres'].fillna('').str.replace(';', ' ')

In [68]:
#@title Criar a Matriz de Características (Vetorização)
cv = CountVectorizer()
matriz_generos = cv.fit_transform(filmes['genres_clean'])

In [69]:
#@title Calcular a Similaridade de Cosseno

# Isso cria uma tabela gigante comparando cada filme com todos os outros
# Valor próximo de 1.0 = Muito parecido | Próximo de 0.0 = Nada a ver
similaridade = cosine_similarity(matriz_generos, matriz_generos)

In [74]:
#@title Função de Recomendação
def recomendar_filmes(nome_filme, n_rec=5, peso_genero=0.8, peso_nota=0.2):
    # Verifica se o filme existe na base unificada
    if nome_filme not in filmes['name'].values:
        return f"Filme '{nome_filme}' não encontrado na base de dados."

    # Pega o ID do filme
    idx = filmes[filmes['name'] == nome_filme].index[0]

    # Calcula scores de similaridade
    sim_scores = list(enumerate(similaridade[idx]))

    # Calcula score híbrido (Similaridade + Nota)
    scores_finais = []
    for i, score_sim in sim_scores:
        if i == idx: continue # Pula o próprio filme

        # Normaliza a nota (0 a 1)
        nota = filmes.iloc[i]['rating']
        nota_norm = nota / 10.0

        # Fórmula Híbrida
        score_final = (score_sim * peso_genero) + (nota_norm * peso_nota)
        scores_finais.append((i, score_final))

    # Ordena e pega os top N
    scores_finais = sorted(scores_finais, key=lambda x: x[1], reverse=True)
    top_indices = [i[0] for i in scores_finais[:n_rec]]

    # Retorna o resultado
    return filmes.iloc[top_indices][['name', 'genres', 'rating', 'year']]

In [81]:
#@title Teste

filme_escolhido = "The Matrix"
print(f"Filmes parecidos com '{filme_escolhido}' (Baseado em Gêneros):")
print(recomendar_filmes(filme_escolhido))

Filmes parecidos com 'The Matrix' (Baseado em Gêneros):
                                    name            genres  rating  year
16            Terminator 2: Judgment Day  Action; Sci-Fi;      8.5  1991
26                        The Terminator  Action; Sci-Fi;      8.0  1984
89                   The Matrix Reloaded  Action; Sci-Fi;      7.2  2003
1069              The Matrix Revolutions  Action; Sci-Fi;      6.8  2003
1088  Terminator 3: Rise of the Machines  Action; Sci-Fi;      6.3  2003
